In [16]:
import pygame
import random
import math

pygame.init()
pygame.joystick.init()  # Initialize the joystick module

# Original window size for scaling reference
OLD_WINDOW_SIZE = (600, 600)

# New window size
WINDOW_SIZE = (1200, 800)

# Scaling factors
SCALING_FACTOR_X = WINDOW_SIZE[0] / OLD_WINDOW_SIZE[0]
SCALING_FACTOR_Y = WINDOW_SIZE[1] / OLD_WINDOW_SIZE[1]
SCALING_FACTOR = (SCALING_FACTOR_X + SCALING_FACTOR_Y) / 2  # Average scaling factor

# Constants (adjusted with scaling factor)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 200, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
LIGHT_GREEN = (144, 238, 144)  # For path prediction
FONT_COLOR = (0, 0, 0)
FONT_SIZE = int(24 * SCALING_FACTOR)
ARROW_LENGTH = int(60 * SCALING_FACTOR)
NUM_GOALS = 3
MAX_DISTANCE_FROM_HUMAN = int(100 * SCALING_FACTOR)
current_noise_value = 0

# Hyperparameters
NUM_OBSTACLES = 5  # Number of obstacles
ENABLE_OBSTACLES = True  # Set to False to disable obstacle generation
OBSTACLE_RADIUS = int(20 * SCALING_FACTOR)  # Radius of obstacles
COLLISION_BUFFER = int(5 * SCALING_FACTOR)  # Extra buffer for collision detection
GRAY = (128, 128, 128)  # Color for obstacles
NOISE_MAGNITUDE = 1

# Global variables
obstacles = []

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Path Prediction")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Dot position and directions
START_POS = [WINDOW_SIZE[0] // 2, WINDOW_SIZE[1] // 2]
dot_pos = START_POS.copy()  # The dot's position
gamma = 0.5  # Arbitration function
reached_goal = False

# Maximum speed
MAX_SPEED = 3 * SCALING_FACTOR  # Slowed down from 5 to 3

# Create multiple targets
targets = []
for _ in range(NUM_GOALS):
    targets.append([random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])])

current_target_idx = 0

# Initialize joystick
joystick = None
if pygame.joystick.get_count() > 0:
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print("Joystick initialized:", joystick.get_name())
else:
    print("No joystick detected.")

def generate_obstacles():
    """Generate non-overlapping obstacles"""
    obstacles.clear()
    if not ENABLE_OBSTACLES:
        return  # Do not generate obstacles if disabled
    for _ in range(NUM_OBSTACLES):
        while True:
            pos = [random.randint(OBSTACLE_RADIUS, WINDOW_SIZE[0]-OBSTACLE_RADIUS),
                  random.randint(OBSTACLE_RADIUS, WINDOW_SIZE[1]-OBSTACLE_RADIUS)]
            if distance(pos, dot_pos) > OBSTACLE_RADIUS * 2:
                overlap = False
                for other_pos in obstacles:
                    if distance(pos, other_pos) < OBSTACLE_RADIUS * 2.5:
                        overlap = True
                        break
                if not overlap:
                    obstacles.append(pos)
                    break

def generate_targets():
    """Generate targets that don't overlap with obstacles"""
    targets.clear()
    for _ in range(NUM_GOALS):
        while True:
            pos = [random.randint(0, WINDOW_SIZE[0]),
                  random.randint(0, WINDOW_SIZE[1])]
            valid_position = True
            if ENABLE_OBSTACLES:
                for obstacle_pos in obstacles:
                    if distance(pos, obstacle_pos) < OBSTACLE_RADIUS * 1.5:
                        valid_position = False
                        break
            if valid_position:
                targets.append(pos)
                break

def distance(pos1, pos2):
    return math.hypot(pos1[0] - pos2[0], pos1[1] - pos2[1])

def normalize_vector(dx, dy):
    length = math.hypot(dx, dy)
    if length == 0:
        return [0, 0]
    return [dx / length, dy / length]

def check_collision(pos, new_pos):
    """Check if moving to new_pos would result in a collision with any obstacle"""
    if not ENABLE_OBSTACLES:
        return False  # No collision if obstacles are disabled
    for obstacle_pos in obstacles:
        if line_circle_intersection(pos, new_pos, obstacle_pos, OBSTACLE_RADIUS + COLLISION_BUFFER):
            return True
    return False

def line_circle_intersection(start, end, circle_center, radius):
    """Check if a line segment intersects with a circle"""
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    cx = circle_center[0] - start[0]
    cy = circle_center[1] - start[1]
    l2 = dx*dx + dy*dy
    if l2 == 0:
        return distance(start, circle_center) <= radius
    t = max(0, min(1, (cx*dx + cy*dy) / l2))
    projection_x = start[0] + t * dx
    projection_y = start[1] + t * dy
    return distance([projection_x, projection_y], circle_center) <= radius

def predict_human_target(human_input):
    global current_target_idx

    if human_input[0] == 0 and human_input[1] == 0:
        return current_target_idx

    best_score = float('-inf')
    best_target_idx = current_target_idx

    for i, target in enumerate(targets):
        to_target_dx = target[0] - dot_pos[0]
        to_target_dy = target[1] - dot_pos[1]

        movement_mag = math.hypot(human_input[0], human_input[1])
        target_mag = math.hypot(to_target_dx, to_target_dy)

        if movement_mag == 0 or target_mag == 0:
            continue

        alignment = (human_input[0] * to_target_dx + human_input[1] * to_target_dy) / (movement_mag * target_mag)

        dist = distance(dot_pos, target)
        max_dist = math.hypot(WINDOW_SIZE[0], WINDOW_SIZE[1])
        distance_factor = 1 - (dist / max_dist)

        score = (alignment * 0.7) + (distance_factor * 0.3)

        if score > best_score:
            best_score = score
            best_target_idx = i

    return best_target_idx

def move_dot(human_input):
    global dot_pos, gamma, reached_goal, current_target_idx

    # Calculate human movement vector (blue)
    h_dx, h_dy = human_input
    h_mag = math.hypot(h_dx, h_dy)
    if h_mag > 0:
        h_dir = [h_dx / h_mag, h_dy / h_mag]
    else:
        h_dir = [0, 0]

    # Calculate perfect path vector (green)
    target_pos = targets[current_target_idx]
    w_dx = target_pos[0] - dot_pos[0]
    w_dy = target_pos[1] - dot_pos[1]
    w_mag = math.hypot(w_dx, w_dy)
    if w_mag > 0:
        w_dir = [w_dx / w_mag, w_dy / w_mag]
    else:
        w_dir = [0, 0]

    # Calculate combined movement vector (red)
    x_dir_x = gamma * w_dir[0] + (1 - gamma) * h_dir[0]
    x_dir_y = gamma * w_dir[1] + (1 - gamma) * h_dir[1]
    x_dir_mag = math.hypot(x_dir_x, x_dir_y)
    if x_dir_mag > 0:
        x_dir = [x_dir_x / x_dir_mag, x_dir_y / x_dir_mag]
    else:
        x_dir = [0, 0]

    # Scale the movement by the magnitude of the human input
    input_mag = h_mag / MAX_SPEED  # Normalize input magnitude
    input_mag = min(max(input_mag, 0), 1)  # Clamp between 0 and 1

    # Move the dot
    step_size = MAX_SPEED * input_mag
    new_x = dot_pos[0] + x_dir[0] * step_size
    new_y = dot_pos[1] + x_dir[1] * step_size

    # Check for collision before moving
    if not check_collision(dot_pos, [new_x, new_y]):
        dot_pos[0] = max(0, min(WINDOW_SIZE[0], new_x))
        dot_pos[1] = max(0, min(WINDOW_SIZE[1], new_y))

    # Check if reached the target
    if distance(dot_pos, target_pos) < OBSTACLE_RADIUS:
        if current_target_idx == len(targets) - 1:
            reached_goal = True
            pygame.time.set_timer(pygame.USEREVENT, 1000)
        else:
            current_target_idx += 1

    return h_dir, w_dir, x_dir

def reset():
    global dot_pos, reached_goal, current_target_idx, gamma

    dot_pos = START_POS.copy()
    reached_goal = False
    current_target_idx = 0
    gamma = 0.5

    generate_obstacles()  # Generate new obstacles first
    generate_targets()    # Then generate targets that avoid obstacles

    # Clear any pending auto-reset events
    pygame.time.set_timer(pygame.USEREVENT, 0)

def draw_arrow(surface, color, start_pos, direction, length=ARROW_LENGTH):
    dx, dy = direction
    if dx == 0 and dy == 0:
        return

    # Normalize the direction
    dir_length = math.hypot(dx, dy)
    dx /= dir_length
    dy /= dir_length

    end_x = start_pos[0] + dx * length
    end_y = start_pos[1] + dy * length

    pygame.draw.line(surface, color, start_pos, (end_x, end_y), int(2 * SCALING_FACTOR))

    arrow_size = 7 * SCALING_FACTOR
    angle = math.atan2(dy, dx)
    arrow1_x = end_x - arrow_size * math.cos(angle + math.pi/6)
    arrow1_y = end_y - arrow_size * math.sin(angle + math.pi/6)
    arrow2_x = end_x - arrow_size * math.cos(angle - math.pi/6)
    arrow2_y = end_y - arrow_size * math.sin(angle - math.pi/6)

    pygame.draw.line(surface, color, (end_x, end_y), (arrow1_x, arrow1_y), int(2 * SCALING_FACTOR))
    pygame.draw.line(surface, color, (end_x, end_y), (arrow2_x, arrow2_y), int(2 * SCALING_FACTOR))

def render(h_dir, w_dir, x_dir):
    screen.fill(WHITE)

    # Draw obstacles
    if ENABLE_OBSTACLES:
        for obstacle_pos in obstacles:
            pygame.draw.circle(screen, GRAY, (int(obstacle_pos[0]), int(obstacle_pos[1])), OBSTACLE_RADIUS)

    # Draw all targets with numbers
    for i, target in enumerate(targets):
        pygame.draw.circle(screen, YELLOW, (int(target[0]), int(target[1])), int(10 * SCALING_FACTOR))
        num_text = font.render(str(i+1), True, BLACK)
        screen.blit(num_text, (target[0] - 5, target[1] - 12))

    # Highlight current target with a ring
    current_target = targets[current_target_idx]
    pygame.draw.circle(screen, BLACK, (int(current_target[0]), int(current_target[1])), int(12 * SCALING_FACTOR), int(2 * SCALING_FACTOR))

    # Draw big empty circle at dot's position
    pygame.draw.circle(screen, BLACK, (int(dot_pos[0]), int(dot_pos[1])), int(30 * SCALING_FACTOR), int(2 * SCALING_FACTOR))

    # Draw human movement vector (blue)
    if h_dir[0] != 0 or h_dir[1] != 0:
        draw_arrow(screen, BLUE, (int(dot_pos[0]), int(dot_pos[1])), h_dir, length=ARROW_LENGTH)

    # Draw perfect path vector (green)
    if w_dir[0] != 0 or w_dir[1] != 0:
        draw_arrow(screen, GREEN, (int(dot_pos[0]), int(dot_pos[1])), w_dir, length=ARROW_LENGTH)

    # Draw combined movement vector (red)
    if x_dir[0] != 0 or x_dir[1] != 0:
        draw_arrow(screen, RED, (int(dot_pos[0]), int(dot_pos[1])), x_dir, length=ARROW_LENGTH)

    # Display information
    gamma_text = font.render(f"Gamma: {gamma:.2f}", True, FONT_COLOR)
    screen.blit(gamma_text, (10, 10))
    formula_text = font.render(f"Movement = {gamma:.2f}W + {1-gamma:.2f}H", True, FONT_COLOR)
    screen.blit(formula_text, (10, 40))
    target_text = font.render(f"Current Target: {current_target_idx + 1}", True, FONT_COLOR)
    screen.blit(target_text, (10, 70))

    # Instructions
    instructions_text = font.render("L2/R2 to adjust gamma, R to reset", True, FONT_COLOR)
    screen.blit(instructions_text, (10, 100))

    # Legend for arrows
    legend_y = WINDOW_SIZE[1] - int(100 * SCALING_FACTOR)
    legend_spacing = int(30 * SCALING_FACTOR)
    legend_items = [
        ("Green Arrow: Perfect Path (W)", GREEN),
        ("Blue Arrow: Human Movement (H)", BLUE),
        ("Red Arrow: Dot's Movement", RED)
    ]
    for i, (text, color) in enumerate(legend_items):
        label = font.render(text, True, color)
        screen.blit(label, (10, legend_y + i * legend_spacing))

    if reached_goal:
        reset_text = font.render("Goal Reached! Auto-resetting...", True, FONT_COLOR)
        screen.blit(reset_text, (150, 160))

    pygame.display.update()

# Main game loop
running = True
clock = pygame.time.Clock()
gamma = 0.5  # Initial gamma value

# Initial setup
generate_obstacles()
generate_targets()

# Axis indices for L2 and R2 (adjust based on your controller)
AXIS_L2 = 2  # L2 trigger
AXIS_R2 = 5  # R2 trigger

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Keyboard reset
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                reset()

        # Controller input
        if joystick:
            # Handle button presses
            if event.type == pygame.JOYBUTTONDOWN:
                if event.button == 2:  # Square button on PS5 controller
                    reset()

        if event.type == pygame.USEREVENT:
            reset()  # Auto-reset when timer expires

    if not reached_goal:
        dx, dy = 0.0, 0.0

        # Keyboard input for movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            dx -= 1.0
        if keys[pygame.K_RIGHT]:
            dx += 1.0
        if keys[pygame.K_UP]:
            dy -= 1.0
        if keys[pygame.K_DOWN]:
            dy += 1.0

        # Joystick input
        if joystick:
            axis_0 = joystick.get_axis(0)  # Left stick horizontal
            axis_1 = joystick.get_axis(1)  # Left stick vertical
            deadzone = 0.1
            if abs(axis_0) > deadzone or abs(axis_1) > deadzone:
                dx = axis_0
                dy = axis_1
            else:
                dx = 0.0
                dy = 0.0

            # Handle L2 and R2 triggers for gamma adjustment
            l2_value = joystick.get_axis(AXIS_L2)
            r2_value = joystick.get_axis(AXIS_R2)

            # L2 and R2 values range from -1 (not pressed) to 1 (fully pressed)
            l2_pressed = l2_value > 0.5
            r2_pressed = r2_value > 0.5

            if l2_pressed:
                gamma = max(0.0, gamma - 0.01)  # Decrease gamma
            if r2_pressed:
                gamma = min(1.0, gamma + 0.01)  # Increase gamma

        # Apply deadzone for keyboard input
        if abs(dx) < 0.1 and abs(dy) < 0.1:
            dx, dy = 0.0, 0.0

        # Scale movement
        dx *= MAX_SPEED
        dy *= MAX_SPEED

        human_input = [dx, dy]
        current_target_idx = predict_human_target(human_input)
        h_dir, w_dir, x_dir = move_dot(human_input)

    else:
        h_dir, w_dir, x_dir = [0, 0], [0, 0], [0, 0]

    render(h_dir, w_dir, x_dir)
    clock.tick(60)

pygame.quit()


Joystick initialized: DualSense Wireless Controller
